# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Для работы мы имеем таблицу с информацией о заемщиках и о том, имели ли они задолженность по возврату кредитов.

**Целями исследования являются ответы на следующие вопросы:**

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

- Как разные цели кредита влияют на его возврат в срок?


## Содержание
2. [Чтение данных](#read_data)
3. [Предобработка данных](#data_processing)
4. [Ответы на вопросы. Анализ](#analysis)
5. [Общий вывод](#conclusion)

## Шаг 1. Чтение данных
<a name='read_data'></a>

Откроем файл с данными и изучим общую информацию:

In [1]:
import pandas as pd

import numpy as np

from pymystem3 import Mystem
m = Mystem()

In [2]:
data = pd.read_csv('/datasets/data.csv')

display(data)

data.info()
print()

display(data.describe())
print()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB



,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [3]:
for column in data:
    print(data[column].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
среднее   

**Вывод**

Изучение данных из таблицы показывает:
- ошибки в данных столбцов children и dob_years,
- отрицательные значения количества дней трудового стажа, что может быть следствием технической ошибки,
- различия в регистре в столбце education,
- пропуски в колонке gender,
- тип данных в колонках days_employed и total_income - вещественное число, а не целое.

При изучении типа данных методом info(), можно заметить, что в столбцах days_employed и total_income данных меньше - там имеются пропуски.

## Шаг 2. Предобработка данных
<a id='data_processing'></a>

### Обработка пропусков

In [4]:
print(data.isna().sum())
print()

print(data[data['days_employed'].isna()].head())
print(len(data[data['days_employed'].isna()]))
print()

print(round(data.isna().sum() * 100 / len(data), 2))

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

    children  days_employed  dob_years education  education_id  \
12         0            NaN         65   среднее             1   
26         0            NaN         41   среднее             1   
29         0            NaN         63   среднее             1   
41         0            NaN         50   среднее             1   
55         0            NaN         54   среднее             1   

            family_status  family_status_id gender  income_type  debt  \
12       гражданский брак                 1      M    пенсионер     0   
26        женат / замужем                 0      M  госслужащий     0   
29  Не женат / не замужем                 4      F    пенсионер     0 

In [5]:
def age_category(dob_years):
    if dob_years < 30:
        return 'до 30'
    if 30 <= dob_years < 40:
        return 'от 30 до 40'  
    if 40 <= dob_years < 50:
        return 'от 40 до 50'
    if dob_years >= 50:
        return 'после 50'

data['age_category'] = data['dob_years'].apply(age_category)

data['days_employed'] = data['days_employed'].fillna(0)

data.loc[(data['days_employed'] == 0) & (data['age_category'] == 'до 30'), 'days_employed'] = data.groupby('age_category')['days_employed'].mean()[0]
data.loc[(data['days_employed'] == 0) & (data['age_category'] == 'от 30 до 40'), 'days_employed'] = data.groupby('age_category')['days_employed'].mean()[1]
data.loc[(data['days_employed'] == 0) & (data['age_category'] == 'от 40 до 50'), 'days_employed'] = data.groupby('age_category')['days_employed'].mean()[2]
data.loc[(data['days_employed'] == 0) & (data['age_category'] == 'после 50'), 'days_employed'] = data.groupby('age_category')['days_employed'].mean()[3]

print(data.loc[12])

display(data.head(15))

children                           0
days_employed                 164006
dob_years                         65
education                    среднее
education_id                       1
family_status       гражданский брак
family_status_id                   1
gender                             M
income_type                пенсионер
debt                               0
total_income                     NaN
purpose              сыграть свадьбу
age_category                после 50
Name: 12, dtype: object


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,от 40 до 50
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,от 30 до 40
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,от 30 до 40
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,от 30 до 40
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,после 50
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,до 30
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,от 40 до 50
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,после 50
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,от 30 до 40
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,от 40 до 50


In [6]:
data['total_income'] = data['total_income'].fillna(0)

data.loc[(data['total_income'] == 0) & (data['income_type'] == 'безработный'), 'total_income'] = data.groupby('income_type')['total_income'].median()[0]
data.loc[(data['total_income'] == 0) & (data['income_type'] == 'в декрете'), 'total_income'] = data.groupby('income_type')['total_income'].median()[1]
data.loc[(data['total_income'] == 0) & (data['income_type'] == 'госслужащий'), 'total_income'] = data.groupby('income_type')['total_income'].median()[2]
data.loc[(data['total_income'] == 0) & (data['income_type'] == 'компаньон'), 'total_income'] = data.groupby('income_type')['total_income'].median()[3]
data.loc[(data['total_income'] == 0) & (data['income_type'] == 'пенсионер'), 'total_income'] = data.groupby('income_type')['total_income'].median()[4]
data.loc[(data['total_income'] == 0) & (data['income_type'] == 'предприниматель'), 'total_income'] = data.groupby('income_type')['total_income'].median()[5]
data.loc[(data['total_income'] == 0) & (data['income_type'] == 'сотрудник'), 'total_income'] = data.groupby('income_type')['total_income'].median()[6]
data.loc[(data['total_income'] == 0) & (data['income_type'] == 'студент'), 'total_income'] = data.groupby('income_type')['total_income'].median()[7]

print(data.loc[12])

children                           0
days_employed                 164006
dob_years                         65
education                    среднее
education_id                       1
family_status       гражданский брак
family_status_id                   1
gender                             M
income_type                пенсионер
debt                               0
total_income                  110180
purpose              сыграть свадьбу
age_category                после 50
Name: 12, dtype: object


In [7]:
data['children'] = abs(data['children'])
data['children'] = data['children'].replace(20, 2)

print(data['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


In [8]:
print(data['dob_years'].median())
data['dob_years'] = data['dob_years'].replace(0, 42)
print()

print(data['dob_years'].value_counts())

42.0

42    698
35    617
40    609
41    607
34    603
38    598
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


In [9]:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [10]:
data['gender'] = data['gender'].replace('XNA', 'F')
print(data['gender'].value_counts())

F    14237
M     7288
Name: gender, dtype: int64


**Вывод**

Пропуски встречаются в 2174 случаях в 2-х столбцах, и это 10% от всех значений датафрейма, в котором 21525 строк, поэтому удаление пропусков может повлиять на результаты исследования. Данные могли быть утеряны при обновлении базы данных или в результате ошибки при внесении в базу данных.

В столбцах days_employed и total_income находятся количественные значения, среди которых есть выдающиеся, т.ч. лучше будет заполнить пропуски медианой значений.

Для total_income, который зависит от типа занятости, сначала заполнили отсутствующие значения нулями, потом сгруппировали каждую категорию по условиям и посчитали медиану.

Вывели на экран строку для проверки (для примера, с индексом 12), а также снова проверили наличие пропусков.

В колонке children убрали отрицательные значения и заменили некорректное количество.

В столбце dob_years заменили пропуски медианой значений.

Столбец education привели к одному регистру.

В колонке gender заменили отсутствующее значение на наиболее частотное.

### Замена типа данных

Попробуем заменить тип данных двумя способами (astype('int') и np.int32) и замерим, сколько памяти займут данные после преобразования. Второй способ позволяет занять меньше памяти - оставим его.

In [11]:
# Первый способ:

# data['days_employed'] = abs(data['days_employed'].astype('int'))
# data['total_income'] = data['total_income'].astype('int')
# display(data.head(10))

# def mem_usage(pandas_obj):
#     if isinstance(pandas_obj,pd.DataFrame):
#         usage_b = pandas_obj.memory_usage(deep=True).sum()
#     else:
#         usage_b = pandas_obj.memory_usage(deep=True)
#     usage_mb = usage_b / 1024 ** 2
#     return "{:03.2f} MB".format(usage_mb)

# print(mem_usage(data['total_income']))
# print(mem_usage(data['days_employed']))

In [12]:
# Второй способ:

data['total_income'] = np.int32(data['total_income'])
data['days_employed'] = abs(np.int32(data['days_employed']))

display(data.head(10))
data.info()
print()

def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else:
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2
    return "{:03.2f} MB".format(usage_mb)

print(mem_usage(data['total_income']))
print(mem_usage(data['days_employed']))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,от 40 до 50
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,от 30 до 40
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,от 30 до 40
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,от 30 до 40
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,после 50
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,до 30
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,от 40 до 50
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,после 50
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,от 30 до 40
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,от 40 до 50


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int32
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int32
purpose             21525 non-null object
age_category        21525 non-null object
dtypes: int32(2), int64(5), object(6)
memory usage: 2.0+ MB

0.08 MB
0.08 MB


**Вывод**

Методы astype('int') и np.int32 подходят для перевода вещественного числа в целое. А отрицательные значения можно убрать с помощью функции abs.

А чтобы сэкономить объем памяти, можно использовать библиотеку NumPy.

### Обработка дубликатов

In [13]:
print(data.duplicated().sum())
print()

data = data.drop_duplicates().reset_index(drop=True)

print(data.duplicated().sum())

71

0


**Вывод**

Удалили 72 дубликата, перезаписав все индексы.

Причиной появления дубликатов могли быть ошибки ввода, разный формат данных.

### Лемматизация

In [14]:
def purpose_lemma(purpose):
    lemmas = ' '.join(m.lemmatize(purpose))
    return lemmas

data['purpose_lemma'] = data['purpose'].apply(purpose_lemma)

display(data['purpose_lemma'].unique())

display(data)

array(['покупка   жилье \n', 'приобретение   автомобиль \n',
       'дополнительный   образование \n', 'сыграть   свадьба \n',
       'операция   с   жилье \n', 'образование \n',
       'на   проведение   свадьба \n', 'покупка   жилье   для   семья \n',
       'покупка   недвижимость \n',
       'покупка   коммерческий   недвижимость \n',
       'покупка   жилой   недвижимость \n',
       'строительство   собственный   недвижимость \n', 'недвижимость \n',
       'строительство   недвижимость \n',
       'на   покупка   подержать   автомобиль \n',
       'на   покупка   свой   автомобиль \n',
       'операция   с   коммерческий   недвижимость \n',
       'строительство   жилой   недвижимость \n', 'жилье \n',
       'операция   со   свой   недвижимость \n', 'автомобиль \n',
       'заниматься   образование \n',
       'сделка   с   подержанный   автомобиль \n',
       'получение   образование \n', 'свадьба \n',
       'получение   дополнительный   образование \n',
       'покупка   свой 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,purpose_lemma
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,от 40 до 50,покупка жилье \n
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,от 30 до 40,приобретение автомобиль \n
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,от 30 до 40,покупка жилье \n
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,от 30 до 40,дополнительный образование \n
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,после 50,сыграть свадьба \n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,от 40 до 50,операция с жилье \n
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,после 50,сделка с автомобиль \n
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,от 30 до 40,недвижимость \n
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,от 30 до 40,на покупка свой автомобиль \n


**Вывод**

Для столбца с целями получения кредита написали функцию, которая приводит слова к их словарной форме, чтобы по ним можно было сделать категоризацию данных. Для этого использовали библиотеку pymystem3.

Создали столбец purpose_lemma.

Вывели уникальные значения.

### Категоризация данных

In [15]:
def purpose_category(purpose):
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'образование' in purpose:
        return 'образование'
    if 'свадьба' in purpose:
        return 'свадьба'
    if 'недвижимость' in purpose:
        return 'недвижимость'
    if 'строительство' in purpose:
        return 'строительство'
    if 'жилье' in purpose:
        return 'жилье'

data['purpose_category'] = data['purpose_lemma'].apply(purpose_category)

display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,purpose_lemma,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,от 40 до 50,покупка жилье \n,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,от 30 до 40,приобретение автомобиль \n,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,от 30 до 40,покупка жилье \n,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,от 30 до 40,дополнительный образование \n,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,после 50,сыграть свадьба \n,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,от 40 до 50,операция с жилье \n,жилье
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,после 50,сделка с автомобиль \n,автомобиль
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,от 30 до 40,недвижимость \n,недвижимость
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,от 30 до 40,на покупка свой автомобиль \n,автомобиль


**Вывод**

Написали функцию и создали столбец purpose_category для отображения категорий в зависимости от целей кредита по словам из колонки purpose_lemma.

## Шаг 3. Ответы на вопросы. Анализ
<a id='analysis'></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [16]:
def child_category(children):
    if children == 0:
        return 0
    if children == 1:
        return 1
    if children == 2:
        return 2
    return 'более 2-х'

data['child_category'] = data['children'].apply(child_category)

display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,purpose_lemma,purpose_category,child_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,от 40 до 50,покупка жилье \n,жилье,1
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,от 30 до 40,приобретение автомобиль \n,автомобиль,1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,от 30 до 40,покупка жилье \n,жилье,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,от 30 до 40,дополнительный образование \n,образование,более 2-х
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,после 50,сыграть свадьба \n,свадьба,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,от 40 до 50,операция с жилье \n,жилье,1
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,после 50,сделка с автомобиль \n,автомобиль,0
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,от 30 до 40,недвижимость \n,недвижимость,1
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,от 30 до 40,на покупка свой автомобиль \n,автомобиль,более 2-х


In [17]:
data_pivot_child = data.pivot_table(index=['child_category'], columns='debt', values='days_employed', aggfunc='count')
print(data_pivot_child)
print()

data_pivot_child['ratio'] = data_pivot_child[1] / data_pivot_child[0] * 100
print(data_pivot_child)

debt                0     1
child_category             
0               13028  1063
1                4410   445
2                1926   202
более 2-х         349    31

debt                0     1      ratio
child_category                        
0               13028  1063   8.159349
1                4410   445  10.090703
2                1926   202  10.488058
более 2-х         349    31   8.882521


**Вывод**

Объединили заемщиков с количеством детей более двух в одну категорию - child_category.

Создали сводную таблицу для наглядного представления данных и добавили столбец ratio со значением соотношения в процентах.

*Итог:* чем больше детей в семье, тем больше вероятность того, что кредит не будет выплачен в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [18]:
data_pivot_family = data.pivot_table(index=['family_status'], columns='debt', values='days_employed', aggfunc='count')

data_pivot_family['ratio'] = data_pivot_family[1] / data_pivot_family[0] * 100
print(data_pivot_family)

debt                       0    1      ratio
family_status                               
Не женат / не замужем   2536  274  10.804416
в разводе               1110   85   7.657658
вдовец / вдова           896   63   7.031250
гражданский брак        3763  388  10.310922
женат / замужем        11408  931   8.160940


**Вывод**

Создали сводную таблицу со столбцом ratio.

*Итог:* не состоящие в браке или люди в гражданском браке выплачивают кредит в срок реже тех, кто относится к другим группам по семейному положению.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [19]:
def income_category(total_income):
    if total_income < 50000:
        return 'Доход до 50000'
    if 50000 <= total_income < 100000:
        return 'Доход от 50000 до 100000'
    if 100000 <= total_income < 150000:
        return 'Доход от 100000 до 150000'
    if 150000 <= total_income < 200000:
        return 'Доход от 150000 до 200000'
    return 'Доход свыше 200000'
    
data['income_category'] = data['total_income'].apply(income_category)

display(data)
print()

data_pivot_income = data.pivot_table(index=['income_category'], columns='debt', values='days_employed', aggfunc='count')

data_pivot_income['ratio'] = data_pivot_income[1] / data_pivot_income[0] * 100
print(data_pivot_income)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,purpose_lemma,purpose_category,child_category,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,от 40 до 50,покупка жилье \n,жилье,1,Доход свыше 200000
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,от 30 до 40,приобретение автомобиль \n,автомобиль,1,Доход от 100000 до 150000
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,от 30 до 40,покупка жилье \n,жилье,0,Доход от 100000 до 150000
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,от 30 до 40,дополнительный образование \n,образование,более 2-х,Доход свыше 200000
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,после 50,сыграть свадьба \n,свадьба,0,Доход от 150000 до 200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,от 40 до 50,операция с жилье \n,жилье,1,Доход свыше 200000
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,после 50,сделка с автомобиль \n,автомобиль,0,Доход от 150000 до 200000
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,от 30 до 40,недвижимость \n,недвижимость,1,Доход от 50000 до 100000
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,от 30 до 40,на покупка свой автомобиль \n,автомобиль,более 2-х,Доход свыше 200000



debt                          0    1     ratio
income_category                               
Доход до 50000              349   23  6.590258
Доход от 100000 до 150000  6674  631  9.454600
Доход от 150000 до 200000  4221  398  9.429045
Доход от 50000 до 100000   3760  331  8.803191
Доход свыше 200000         4709  358  7.602463


**Вывод**

Разделили уровни доходов на категории с помощью функции и добавили столбец income_category.

*Итог:* при доходе от 100 до 200 тыс. просрочки случаются чаще, чем в случаях более низких доходов. При доходе от 200 тыс. просрочек становится меньше, но их всё же больше по сравнению с доходами до 50 тыс.

- Как разные цели кредита влияют на его возврат в срок?

In [20]:
data_pivot_purpose = data.pivot_table(index=['purpose_category'], columns='debt', values='days_employed', aggfunc='count')

data_pivot_purpose['ratio'] = data_pivot_purpose[1] / data_pivot_purpose[0] * 100
print(data_pivot_purpose)

debt                 0    1      ratio
purpose_category                      
автомобиль        3903  403  10.325391
жилье             4152  308   7.418112
недвижимость      5877  474   8.065339
образование       3643  370  10.156464
свадьба           2138  186   8.699719


**Вывод**

Создали сводную таблицу со столбцом ratio.

*Итог:* кредиты на автомобиль и образование реже выплачиваются в срок.

## Шаг 4. Общий вывод
<a id='conclusion'></a>

В таблице обработаны пропуски, дубликаты, артефакты. Добавлены столбцы с категориями для группировки данных и подсчета значений.

Результаты исследования дали ответы на поставленные вопросы:

**• Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Чем больше детей в семье, тем больше вероятность того, что кредит не будет выплачен в срок, о чем свидетельствует значение соотношения наличия и отсутствия задолженности в зависимости от количества детей. Чем больше значение в столбце “ratio”, тем выше вероятность:


In [21]:
print(data_pivot_child)

debt                0     1      ratio
child_category                        
0               13028  1063   8.159349
1                4410   445  10.090703
2                1926   202  10.488058
более 2-х         349    31   8.882521


**• Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Не состоящие в браке или люди в гражданском браке выплачивают кредит в срок реже тех, кто относится к другим группам по семейному положению:

In [22]:
print(data_pivot_family)

debt                       0    1      ratio
family_status                               
Не женат / не замужем   2536  274  10.804416
в разводе               1110   85   7.657658
вдовец / вдова           896   63   7.031250
гражданский брак        3763  388  10.310922
женат / замужем        11408  931   8.160940


**• Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

При доходе от 100 до 200 тыс. просрочки случаются чаще, чем в случаях более низких доходов. При доходе от 200 тыс. просрочек становится меньше, но их всё же больше по сравнению с доходами до 50 тыс.: 

In [23]:
print(data_pivot_income)

debt                          0    1     ratio
income_category                               
Доход до 50000              349   23  6.590258
Доход от 100000 до 150000  6674  631  9.454600
Доход от 150000 до 200000  4221  398  9.429045
Доход от 50000 до 100000   3760  331  8.803191
Доход свыше 200000         4709  358  7.602463


**• Как разные цели кредита влияют на его возврат в срок?**

Кредиты на автомобиль и образование реже выплачиваются в срок:

In [24]:
print(data_pivot_purpose)

debt                 0    1      ratio
purpose_category                      
автомобиль        3903  403  10.325391
жилье             4152  308   7.418112
недвижимость      5877  474   8.065339
образование       3643  370  10.156464
свадьба           2138  186   8.699719


**Итоги:**

На выплату кредита в срок влияют различные факторы, такие как наличие и количество детей в семье, семейное положение, уровень дохода и цели кредита.